In [30]:
import pandas as pd
spot = pd.read_csv('./data/spot/clarkson_data.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
pmx_forw = pd.read_csv('./data/ffa/PMAX_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
csz_forw = pd.read_csv('./data/ffa/CSZ_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
smx_forw = pd.read_csv('./data/ffa/SMX_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)


In [31]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Merge data frames on the Date column
data_combined = pd.merge(spot, smx_forw, on='Date')
s_col = "SMX"
f_col = "1Q"

# Remove rows with NA or 0 in specific columns (assuming 'SMX' and '1Q' are column names in 'data_combined')
data_combined = data_combined[(data_combined[s_col].notna() & data_combined[s_col] != 0) & (data_combined[f_col].notna() & data_combined[f_col] != 0)]

# Transform data to log levels
data_log_levels = pd.DataFrame()
data_log_levels["spot"] = np.log(data_combined[s_col])
data_log_levels["forwp"] = np.log(data_combined[f_col])
data_log_levels.index = data_combined["Date"]


# Split into train and test sets
split_index = round(len(data_log_levels) * 0.8)
hor = 30
train = data_log_levels.iloc[:split_index]
test = data_log_levels.iloc[split_index:split_index+hor]
#train.head()
data_log_levels.head()
train.head()
#test.head()

# Normalize features
scaler = MinMaxScaler()
train_scal = scaler.fit_transform(train)
print(train_scal)


[[0.58835698 0.46232689]
 [0.58560421 0.46323447]
 [0.56566024 0.43324572]
 ...
 [0.39615883 0.29126376]
 [0.40421298 0.28693668]
 [0.40808548 0.2882623 ]]


In [32]:
# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=10, is_test=False):
    X, Y = [], []
    if is_test:  # for test data, we just need the last entry for 1-step ahead forecast
        X = dataset[-1:,:,]
        return X, None
    else:
        for i in range(look_back, len(dataset)- hor + 1):
            X.append(dataset[i-look_back:i])
            Y.append(dataset[i:i+hor])
    return np.array(X), np.array(Y)

look_back = 20  # Adjust based on your temporal structure
trainX, trainY = create_dataset(train_scal, look_back)
print(trainX)

[[[0.58835698 0.46232689]
  [0.58560421 0.46323447]
  [0.56566024 0.43324572]
  ...
  [0.58151033 0.40589457]
  [0.58443316 0.41507072]
  [0.59184327 0.43141927]]

 [[0.58560421 0.46323447]
  [0.56566024 0.43324572]
  [0.56119228 0.43027925]
  ...
  [0.58443316 0.41507072]
  [0.59184327 0.43141927]
  [0.59970273 0.44814931]]

 [[0.56566024 0.43324572]
  [0.56119228 0.43027925]
  [0.55692839 0.42903601]
  ...
  [0.59184327 0.43141927]
  [0.59970273 0.44814931]
  [0.60193977 0.43988099]]

 ...

 [[0.20266659 0.24306369]
  [0.19326445 0.25687326]
  [0.19606777 0.2571137 ]
  ...
  [0.27301676 0.10205481]
  [0.23433078 0.21148978]
  [0.22068284 0.2137549 ]]

 [[0.19326445 0.25687326]
  [0.19606777 0.2571137 ]
  [0.20555977 0.26527338]
  ...
  [0.23433078 0.21148978]
  [0.22068284 0.2137549 ]
  [0.20941293 0.20884436]]

 [[0.19606777 0.2571137 ]
  [0.20555977 0.26527338]
  [0.2159535  0.27171733]
  ...
  [0.22068284 0.2137549 ]
  [0.20941293 0.20884436]
  [0.20302236 0.20144473]]]


In [33]:
# Create and fit the MLP model
from keras.layers import Dense
from keras import Sequential
from sklearn.metrics import mean_squared_error
trainX_flat = trainX.reshape(trainX.shape[0], -1)
trainY_flat = trainY.reshape(trainY.shape[0], -1)

model_mlp = Sequential()
model_mlp.add(Dense(32, input_dim=trainX_flat.shape[1], activation='relu'))
model_mlp.add(Dense(trainY_flat.shape[1], activation="linear"))
model_mlp.compile(loss='mean_squared_error', optimizer='adam')
model_mlp.fit(trainX_flat, trainY_flat, epochs=1000, batch_size=2, verbose=2)



Epoch 1/1000


c:\Users\mariumbo\master-thesis\env\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


877/877 - 1s - 2ms/step - loss: 0.0184
Epoch 2/1000
877/877 - 1s - 884us/step - loss: 0.0078
Epoch 3/1000
877/877 - 1s - 945us/step - loss: 0.0073
Epoch 4/1000
877/877 - 1s - 993us/step - loss: 0.0069
Epoch 5/1000
877/877 - 1s - 918us/step - loss: 0.0068
Epoch 6/1000
877/877 - 1s - 862us/step - loss: 0.0067
Epoch 7/1000
877/877 - 1s - 869us/step - loss: 0.0065
Epoch 8/1000
877/877 - 1s - 833us/step - loss: 0.0065
Epoch 9/1000
877/877 - 1s - 844us/step - loss: 0.0063
Epoch 10/1000
877/877 - 1s - 845us/step - loss: 0.0061
Epoch 11/1000
877/877 - 1s - 840us/step - loss: 0.0061
Epoch 12/1000
877/877 - 1s - 878us/step - loss: 0.0061
Epoch 13/1000
877/877 - 1s - 852us/step - loss: 0.0060
Epoch 14/1000
877/877 - 1s - 832us/step - loss: 0.0059
Epoch 15/1000
877/877 - 1s - 827us/step - loss: 0.0058
Epoch 16/1000
877/877 - 1s - 844us/step - loss: 0.0058
Epoch 17/1000
877/877 - 1s - 821us/step - loss: 0.0056
Epoch 18/1000
877/877 - 1s - 832us/step - loss: 0.0057
Epoch 19/1000
877/877 - 1s - 821us

In [38]:
# Make predictions
trainPredict_scal_flat = model_mlp.predict(trainX_flat)


def create_even_odd_array(arr):
    """
    Returns an array where the first column contains values from even positions
    and the second column contains values from odd positions of the original array.
    """
    return arr.reshape(-1, 2)



testX, _ = create_dataset(trainX, look_back=look_back, is_test=True)
testX_flat = testX.reshape(testX.shape[0], -1)
testPredict_scal_flat = model_mlp.predict(testX_flat)
testPredict_scal = create_even_odd_array(testPredict_scal_flat)


# Invert predictions
#trainPredict = scaler.inverse_transform(trainPredict_scal)
testPredict = scaler.inverse_transform(testPredict_scal)
print(testPredict[:, 0])

# Calculate mean squared error
testScore = mean_squared_error(test["spot"], testPredict[:,0])
testScoreForw = mean_squared_error(test["forwp"], testPredict[:,1])
print('Test Score spot: %.5f MSE' % (testScore))
print('Test Score forw: %.5f MSE' % (testScoreForw))




def random_walk_predictions(training_data, testing_data):
    """
    Generates Random Walk predictions where the next value is assumed to be the last observed value.
    
    Parameters:
    - training_data: DataFrame containing the training data.
    - testing_data: DataFrame containing the test data.
    
    Returns:
    - predictions: Numpy array containing Random Walk predictions for the test set.
    """
    # Last observed values from the training set
    last_observed_spot = training_data['spot'].iloc[-1]
    last_observed_forwp = training_data['forwp'].iloc[-1]
    
    # Create an array of predictions, each one equal to the last observed values
    predictions = np.tile([last_observed_spot, last_observed_forwp], (len(testing_data), 1))
    
    return predictions

# Generate Random Walk predictions for the test set
rw_predictions = random_walk_predictions(train, test)

# Benchmark Random Walk model by calculating the MSE
rw_testScore_spot = mean_squared_error(test["spot"].values, rw_predictions[:, 0])
rw_testScore_forw = mean_squared_error(test["forwp"].values, rw_predictions[:, 1])

print('Random Walk Test Score spot: %.5f MSE' % (rw_testScore_spot))
print('Random Walk Test Score forw: %.5f MSE' % (rw_testScore_forw))

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[8.491202  8.4791355 8.465288  8.454037  8.449972  8.454738  8.479729
 8.509925  8.544253  8.589169  8.637585  8.698681  8.762257  8.82639
 8.894675  8.95582   9.013305  9.070442  9.120139  9.177554  9.22593
 9.260985  9.288203  9.308158  9.309073  9.315817  9.31979   9.3217325
 9.3205805 9.310349 ]
Test Score spot: 0.21963 MSE
Test Score forw: 0.03658 MSE
Random Walk Test Score spot: 0.00323 MSE
Random Walk Test Score forw: 0.01074 MSE


In [40]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(trainY_flat.shape[1], activation='linear'))  # Assuming multi-step forecasting

model_lstm.compile(loss='mean_squared_error', optimizer='adam')
model_lstm.fit(trainX, trainY_flat, epochs=100, batch_size=32, verbose=2)



Epoch 1/100
55/55 - 3s - 60ms/step - loss: 0.0613
Epoch 2/100
55/55 - 0s - 7ms/step - loss: 0.0096
Epoch 3/100
55/55 - 0s - 7ms/step - loss: 0.0089
Epoch 4/100
55/55 - 0s - 7ms/step - loss: 0.0085
Epoch 5/100
55/55 - 0s - 7ms/step - loss: 0.0080
Epoch 6/100
55/55 - 0s - 7ms/step - loss: 0.0080
Epoch 7/100
55/55 - 0s - 7ms/step - loss: 0.0076
Epoch 8/100
55/55 - 0s - 7ms/step - loss: 0.0074
Epoch 9/100
55/55 - 0s - 7ms/step - loss: 0.0070
Epoch 10/100
55/55 - 0s - 7ms/step - loss: 0.0065
Epoch 11/100
55/55 - 0s - 6ms/step - loss: 0.0069
Epoch 12/100
55/55 - 0s - 7ms/step - loss: 0.0070
Epoch 13/100
55/55 - 0s - 7ms/step - loss: 0.0066
Epoch 14/100
55/55 - 0s - 7ms/step - loss: 0.0065
Epoch 15/100
55/55 - 0s - 7ms/step - loss: 0.0063
Epoch 16/100
55/55 - 0s - 7ms/step - loss: 0.0066
Epoch 17/100
55/55 - 0s - 7ms/step - loss: 0.0061
Epoch 18/100
55/55 - 0s - 7ms/step - loss: 0.0062
Epoch 19/100
55/55 - 0s - 7ms/step - loss: 0.0059
Epoch 20/100
55/55 - 0s - 7ms/step - loss: 0.0061
Epoch 21

In [42]:
# Prepare the last sequence from the training set as the input for the first prediction
testX_last_sequence = train_scal[-look_back:].reshape(1, look_back, train_scal.shape[1])

# Make predictions
testPredict_scal_flat = model_lstm.predict(testX_last_sequence)

# Since you're predicting `hor` steps ahead, you might need to adjust the code to generate
# multiple steps if your LSTM model is set up for single-step predictions.
# For simplicity, this example directly uses the LSTM output for multi-step predictions.

# Invert scaling
testPredict_scal = create_even_odd_array(testPredict_scal_flat)
testPredict = scaler.inverse_transform(testPredict_scal)


# Calculate and print MSE for each target
testScore_spot_lstm = mean_squared_error(test["spot"].iloc[:hor].values, testPredict[:,0])
testScore_forw_lstm = mean_squared_error(test["forwp"].iloc[:hor].values, testPredict[:,1])
print('LSTM Test Score spot: %.5f MSE' % (testScore_spot_lstm))
print('LSTM Test Score forw: %.5f MSE' % (testScore_forw_lstm))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
LSTM Test Score spot: 0.00137 MSE
LSTM Test Score forw: 0.01324 MSE


In [48]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Number of rounds based on the test set size and forecast horizon
num_rounds =  5  # Adjusted to ensure we don't exceed the test set
look_back = 10  # Adjust based on your temporal structure
hor = 5

# Initialize dictionary to store MSE results for each model
mse_results = {
    'MLP_spot': [],
    'MLP_forwp': [],
    'LSTM_spot': [],
    'LSTM_forwp': [],
    'RW_spot': [],
    'RW_forwp': [],
}


def create_even_odd_array(arr):
    """
    Returns an array where the first column contains values from even positions
    and the second column contains values from odd positions of the original array.
    """
    return arr.reshape(-1, 2)


# Adjust train and test sets for each forecast round
for round in range(1, num_rounds + 1):
    print("Round", round)
    # Define new split point for each round
    split_index = split_index + (round - 1) * hor
    
    # Update train and test sets
    train = data_log_levels.iloc[:split_index]
    test = data_log_levels.iloc[split_index:split_index+hor]

    trainX, trainY = create_dataset(train_scal, look_back)
    # Create and fit the MLP model
    from keras.layers import Dense
    from keras import Sequential
    from sklearn.metrics import mean_squared_error
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    trainY_flat = trainY.reshape(trainY.shape[0], -1)

    model_mlp = Sequential()
    model_mlp.add(Dense(32, input_dim=trainX_flat.shape[1], activation='relu'))
    model_mlp.add(Dense(trainY_flat.shape[1], activation="linear"))
    model_mlp.compile(loss='mean_squared_error', optimizer='adam')
    model_mlp.fit(trainX_flat, trainY_flat, epochs=30, batch_size=2, verbose=2)


    # Make predictions
    trainPredict_scal_flat = model_mlp.predict(trainX_flat)
    testX, _ = create_dataset(trainX, look_back=look_back, is_test=True)
    testX_flat = testX.reshape(testX.shape[0], -1)
    testPredict_scal_flat = model_mlp.predict(testX_flat)
    testPredict_scal = create_even_odd_array(testPredict_scal_flat)

    # Invert predictions
    testPredict_mlp = scaler.inverse_transform(testPredict_scal)

    # Calculate mean squared error
    testScore = mean_squared_error(test["spot"], testPredict_mlp[:,0])
    testScoreForw = mean_squared_error(test["forwp"], testPredict_mlp[:,1])
    print('Test Score spot: %.5f MSE' % (testScore))
    print('Test Score forw: %.5f MSE' % (testScoreForw))



    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
    model_lstm.add(LSTM(units=50))
    model_lstm.add(Dense(trainY_flat.shape[1], activation='linear'))  # Assuming multi-step forecasting

    model_lstm.compile(loss='mean_squared_error', optimizer='adam')
    model_lstm.fit(trainX, trainY_flat, epochs=30, batch_size=32, verbose=2)

    # Prepare the last sequence from the training set as the input for the first prediction
    testX_last_sequence = train_scal[-look_back:].reshape(1, look_back, train_scal.shape[1])

    # Make predictions
    testPredict_scal_flat = model_lstm.predict(testX_last_sequence)

    # Since you're predicting `hor` steps ahead, you might need to adjust the code to generate
    # multiple steps if your LSTM model is set up for single-step predictions.
    # For simplicity, this example directly uses the LSTM output for multi-step predictions.

    # Invert scaling
    testPredict_scal = create_even_odd_array(testPredict_scal_flat)
    testPredict_lstm = scaler.inverse_transform(testPredict_scal)


    # Calculate and print MSE for each target
    testScore_spot_lstm = mean_squared_error(test["spot"].iloc[:hor].values, testPredict_lstm[:,0])
    testScore_forw_lstm = mean_squared_error(test["forwp"].iloc[:hor].values, testPredict_lstm[:,1])
    print('LSTM Test Score spot: %.5f MSE' % (testScore_spot_lstm))
    print('LSTM Test Score forw: %.5f MSE' % (testScore_forw_lstm))


    # Random Walk Predictions for comparison
    rw_predictions = random_walk_predictions(train, test)
    
    # Calculate and append MSE for each model for this round
    mse_results['MLP_spot'].append(mean_squared_error(test["spot"], testPredict_mlp[:, 0]))
    mse_results['MLP_forwp'].append(mean_squared_error(test["forwp"], testPredict_mlp[:, 1]))

    mse_results['LSTM_spot'].append(mean_squared_error(test["spot"], testPredict_lstm[:, 0]))
    mse_results['LSTM_forwp'].append(mean_squared_error(test["forwp"], testPredict_lstm[:, 1]))

    mse_results['RW_spot'].append(mean_squared_error(test["spot"], rw_predictions[:, 0]))
    mse_results['RW_forwp'].append(mean_squared_error(test["forwp"], rw_predictions[:, 1]))


print(mse_results)

Round 1
Epoch 1/30


c:\Users\mariumbo\master-thesis\env\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


895/895 - 1s - 2ms/step - loss: 0.0129
Epoch 2/30
895/895 - 1s - 860us/step - loss: 0.0018
Epoch 3/30
895/895 - 1s - 868us/step - loss: 0.0015
Epoch 4/30
895/895 - 1s - 878us/step - loss: 0.0014
Epoch 5/30
895/895 - 1s - 901us/step - loss: 0.0013
Epoch 6/30
895/895 - 1s - 862us/step - loss: 0.0012
Epoch 7/30
895/895 - 1s - 866us/step - loss: 0.0012
Epoch 8/30
895/895 - 1s - 864us/step - loss: 0.0012
Epoch 9/30
895/895 - 1s - 859us/step - loss: 0.0011
Epoch 10/30
895/895 - 1s - 877us/step - loss: 0.0011
Epoch 11/30
895/895 - 1s - 866us/step - loss: 0.0011
Epoch 12/30
895/895 - 1s - 883us/step - loss: 0.0010
Epoch 13/30
895/895 - 1s - 869us/step - loss: 0.0010
Epoch 14/30
895/895 - 1s - 877us/step - loss: 0.0011
Epoch 15/30
895/895 - 1s - 870us/step - loss: 0.0010
Epoch 16/30
895/895 - 1s - 877us/step - loss: 0.0011
Epoch 17/30
895/895 - 1s - 852us/step - loss: 9.6722e-04
Epoch 18/30
895/895 - 1s - 850us/step - loss: 9.9849e-04
Epoch 19/30
895/895 - 1s - 843us/step - loss: 0.0010
Epoch 2

c:\Users\mariumbo\master-thesis\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 - 3s - 56ms/step - loss: 0.0457
Epoch 2/30
56/56 - 0s - 4ms/step - loss: 0.0031
Epoch 3/30
56/56 - 0s - 4ms/step - loss: 0.0028
Epoch 4/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 5/30
56/56 - 0s - 4ms/step - loss: 0.0026
Epoch 6/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 7/30
56/56 - 0s - 4ms/step - loss: 0.0025
Epoch 8/30
56/56 - 0s - 4ms/step - loss: 0.0024
Epoch 9/30
56/56 - 0s - 4ms/step - loss: 0.0024
Epoch 10/30
56/56 - 0s - 4ms/step - loss: 0.0023
Epoch 11/30
56/56 - 0s - 4ms/step - loss: 0.0023
Epoch 12/30
56/56 - 0s - 4ms/step - loss: 0.0021
Epoch 13/30
56/56 - 0s - 4ms/step - loss: 0.0020
Epoch 14/30
56/56 - 0s - 4ms/step - loss: 0.0022
Epoch 15/30
56/56 - 0s - 4ms/step - loss: 0.0021
Epoch 16/30
56/56 - 0s - 4ms/step - loss: 0.0019
Epoch 17/30
56/56 - 0s - 4ms/step - loss: 0.0020
Epoch 18/30
56/56 - 0s - 4ms/step - loss: 0.0017
Epoch 19/30
56/56 - 0s - 4ms/step - loss: 0.0017
Epoch 20/30
56/56 - 0s - 4ms/step - loss: 0.0016
Epoch 21/30
56/56 - 0s - 4ms/step - los

...

56/56 - 3s - 58ms/step - loss: 0.0415
Epoch 2/30
56/56 - 0s - 4ms/step - loss: 0.0032
Epoch 3/30
56/56 - 0s - 4ms/step - loss: 0.0029
Epoch 4/30
56/56 - 0s - 4ms/step - loss: 0.0028
Epoch 5/30
56/56 - 0s - 4ms/step - loss: 0.0028
Epoch 6/30
56/56 - 0s - 4ms/step - loss: 0.0028
Epoch 7/30
56/56 - 0s - 4ms/step - loss: 0.0026
Epoch 8/30
56/56 - 0s - 4ms/step - loss: 0.0024
Epoch 9/30
56/56 - 0s - 4ms/step - loss: 0.0024
Epoch 10/30
56/56 - 0s - 4ms/step - loss: 0.0024
Epoch 11/30
56/56 - 0s - 4ms/step - loss: 0.0025
Epoch 12/30
56/56 - 0s - 4ms/step - loss: 0.0022
Epoch 13/30
56/56 - 0s - 4ms/step - loss: 0.0022
Epoch 14/30
56/56 - 0s - 4ms/step - loss: 0.0022
Epoch 15/30
56/56 - 0s - 4ms/step - loss: 0.0020
Epoch 16/30
56/56 - 0s - 4ms/step - loss: 0.0021
Epoch 17/30
56/56 - 0s - 4ms/step - loss: 0.0019
Epoch 18/30
56/56 - 0s - 4ms/step - loss: 0.0019
Epoch 19/30
56/56 - 0s - 4ms/step - loss: 0.0017
Epoch 20/30
56/56 - 0s - 4ms/step - loss: 0.0017
Epoch 21/30
56/56 - 0s - 4ms/step - los

c:\Users\mariumbo\master-thesis\env\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


895/895 - 1s - 2ms/step - loss: 0.0143
Epoch 2/30
895/895 - 1s - 894us/step - loss: 0.0021
Epoch 3/30
895/895 - 1s - 925us/step - loss: 0.0016
Epoch 4/30
895/895 - 1s - 916us/step - loss: 0.0014
Epoch 5/30
895/895 - 1s - 876us/step - loss: 0.0013
Epoch 6/30
895/895 - 1s - 880us/step - loss: 0.0012
Epoch 7/30
895/895 - 1s - 881us/step - loss: 0.0012
Epoch 8/30
895/895 - 1s - 862us/step - loss: 0.0012
Epoch 9/30
895/895 - 1s - 861us/step - loss: 0.0011
Epoch 10/30
895/895 - 1s - 860us/step - loss: 0.0011
Epoch 11/30
895/895 - 1s - 857us/step - loss: 0.0010
Epoch 12/30
895/895 - 1s - 870us/step - loss: 9.7647e-04
Epoch 13/30
895/895 - 1s - 864us/step - loss: 9.9345e-04
Epoch 14/30
895/895 - 1s - 872us/step - loss: 9.6996e-04
Epoch 15/30
895/895 - 1s - 892us/step - loss: 9.5317e-04
Epoch 16/30
895/895 - 1s - 888us/step - loss: 9.6340e-04
Epoch 17/30
895/895 - 1s - 865us/step - loss: 9.6580e-04
Epoch 18/30
895/895 - 1s - 849us/step - loss: 9.5277e-04
Epoch 19/30
895/895 - 1s - 883us/step - 

c:\Users\mariumbo\master-thesis\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 - 3s - 55ms/step - loss: 0.0400
Epoch 2/30
56/56 - 0s - 4ms/step - loss: 0.0033
Epoch 3/30
56/56 - 0s - 4ms/step - loss: 0.0029
Epoch 4/30
56/56 - 0s - 4ms/step - loss: 0.0030
Epoch 5/30
56/56 - 0s - 4ms/step - loss: 0.0029
Epoch 6/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 7/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 8/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 9/30
56/56 - 0s - 4ms/step - loss: 0.0027
Epoch 10/30
56/56 - 0s - 5ms/step - loss: 0.0024
Epoch 11/30
56/56 - 0s - 5ms/step - loss: 0.0024
Epoch 12/30
56/56 - 0s - 4ms/step - loss: 0.0023
Epoch 13/30
56/56 - 0s - 4ms/step - loss: 0.0021
Epoch 14/30
56/56 - 0s - 5ms/step - loss: 0.0020
Epoch 15/30
56/56 - 0s - 5ms/step - loss: 0.0020
Epoch 16/30
56/56 - 0s - 4ms/step - loss: 0.0019
Epoch 17/30
56/56 - 0s - 4ms/step - loss: 0.0017
Epoch 18/30
56/56 - 0s - 4ms/step - loss: 0.0019
Epoch 19/30
56/56 - 0s - 4ms/step - loss: 0.0016
Epoch 20/30
56/56 - 0s - 4ms/step - loss: 0.0016
Epoch 21/30
56/56 - 0s - 5ms/step - los